<a href="https://colab.research.google.com/github/cmskzhan/helloworld/blob/master/concepts/plot_vwap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# https://polygon.io/docs/get_v2_aggs_ticker__stocksTicker__range__multiplier___timespan___from___to__anchor
import requests
import datetime
import json
import base64
import string
import pandas as pd
import numpy as np
import pickle
'''
Getting polygon ohlc, vwap and volume requires symbol, date range, etc
'''


# BASE_URL = "https://api.polygon.io/v2/aggs/ticker/"
url=r"https://api.polygon.io/v2/aggs"
cccccccx = 'Maya5pro+=='
symbol="GS"
multiplier=1
timespan="day"
from_date="2019-01-01"
to_date=datetime.date.today()
vw_url = f"/ticker/{symbol}/range/{multiplier}/{timespan}/{from_date}/{to_date}"
cccccccz = cccccccx + string.ascii_letters[:1:-3] + string.hexdigits[6:12:2]
full_url = url + vw_url
pd.set_option("display.precision", 4)

column_names = { 
"v": "Volume",
"vw": "VWAP",
"o": "Open",
"c": "Adj Close",
"h": "High",
"l": "Low",
"n": "transactions"
  }


In [32]:
def dccc(key, ring):
    ring = base64.urlsafe_b64decode(ring + b'===')
    ring = ring.decode('latin')
    encoded_chars = []
    for i in range(len(ring)):
        key_c = key[i % len(key)]
        encoded_c = chr((ord(ring[i]) - ord(key_c) + 256) % 256)
        encoded_chars.append(encoded_c)
    e_string = ''.join(encoded_chars)
    return e_string

AK = dccc(cccccccz, pickle.loads(requests.get("https://github.com/cmskzhan/helloworld/raw/master/pgon.pik").content))
data = { "apikey": AK, "sort": "asc", "limit": 50000 }

In [33]:

response = requests.get(full_url, data)

parsed = json.loads(response.text)

df_polygon = pd.DataFrame.from_dict(parsed['results'])
df_polygon["Date"] = pd.to_datetime(df_polygon['t']/1000, unit='s').dt.date
df_polygon = df_polygon.rename(columns=column_names)
df_polygon.set_index('Date', inplace=True)
df_polygon.index=pd.to_datetime(df_polygon.index)
df_polygon.drop('t', axis=1, inplace=True)
df_polygon

,Volume,VWAP,Open,Adj Close,High,Low,transactions
Date,,,,,,,
2019-07-05,1511645.0,207.4727,206.86,207.90,208.76,206.2000,19502
2019-07-08,1854058.0,205.9161,206.36,205.75,207.25,204.7050,23745
2019-07-09,1812386.0,206.4147,203.75,207.77,207.94,203.5400,20586
2019-07-10,1779950.0,206.7412,207.18,205.97,208.99,205.6500,19166
2019-07-11,3691672.0,210.3986,206.20,211.35,212.59,206.0000,38725
...,...,...,...,...,...,...,...
2021-06-28,1854688.0,367.4048,367.80,368.71,368.87,363.8600,40035
2021-06-29,2423805.0,374.2159,374.86,372.62,378.09,370.7495,50908
2021-06-30,2266149.0,377.6343,370.90,379.53,380.11,370.9000,45613


In [34]:

def VWAP_WSTD_BB(df, span_window, std_weight):
    df['VWAP_EMA'] = df['VWAP'].ewm(span=span_window, min_periods=span_window/2).mean() #draw a VWAP EMA line
    df['VWAP_WSTD'] = df['VWAP_EMA'].rolling(window=span_window).std()*std_weight # calculate sigma (std) * weight

    return df['VWAP_EMA'] + df['VWAP_WSTD'], df['VWAP_EMA'] - df['VWAP_WSTD']

# df_polygon['VWAP_EMA26'] = df_polygon['VWAP'].ewm(span=26, min_periods=10).mean()
# df_polygon['VWAP_WSTD'] = df_polygon['VWAP_EMA26'].rolling(window=26).std()*2.2

UPPER, LOWER = VWAP_WSTD_BB(df_polygon, 26, 2.2)
df_polygon.tail(10)

,Volume,VWAP,Open,Adj Close,High,Low,transactions,VWAP_EMA,VWAP_WSTD
Date,,,,,,,,,
2021-06-21,2778594.0,355.7188,352.89,357.54,357.9700,351.0400,55882,368.1267,16.5297
2021-06-22,1911381.0,356.9185,357.00,357.53,359.8800,352.6400,43814,367.2965,15.6942
2021-06-23,1745242.0,360.5493,358.79,361.05,362.4800,357.6600,40185,366.7967,14.8714
2021-06-24,2715898.0,367.4342,363.00,368.75,370.8800,362.7800,52389,366.8439,14.0743
2021-06-25,2822275.0,368.7083,368.82,368.77,371.1499,364.5400,49862,366.9820,13.1809
2021-06-28,1854688.0,367.4048,367.80,368.71,368.8700,363.8600,40035,367.0133,12.1871
2021-06-29,2423805.0,374.2159,374.86,372.62,378.0900,370.7495,50908,367.5469,11.1741
2021-06-30,2266149.0,377.6343,370.90,379.53,380.1100,370.9000,45613,368.2941,10.1756
2021-07-01,2602048.0,376.2570,380.70,374.99,381.6400,374.1450,56602,368.8839,9.1666


In [35]:
import plotly.graph_objects as go
# 准备画布
fig = go.Figure()

fig = go.Figure(data=go.Ohlc(x=df_polygon.index,
                    open=df_polygon['Open'],
                    high=df_polygon['High'],
                    low=df_polygon['Low'],
                    close=df_polygon['Adj Close'],
                    name='K-chart'))


fig.add_trace(go.Scatter(  # 
    x=UPPER.index, 
    y=UPPER,               
    mode='lines', # line mode          
    name='UPPER Band',
    line=dict(color='royalblue', width=0.5))) # or color='#00ffff'

fig.add_trace(go.Scatter(  # 
    x=df_polygon.index, 
    y=LOWER,               
    mode='lines', # line mode              
    name='LOWER Band',
    line=dict(color='firebrick', width=0.5))) # or color='#0000ff'

fig.show()

Initial stop loss ratio analysis


In [36]:
daily_volatility = (df_polygon['High'] - df_polygon['Low'])/df_polygon['Low']
initial_dailyVol_pertentage = np.percentile(daily_volatility, [70, 75, 80]).mean()

In [37]:
df_hl = df_polygon[['High','Low']]
df_hl.head(20)

,High,Low
Date,,
2019-07-05,208.760,206.200
2019-07-08,207.250,204.705
2019-07-09,207.940,203.540
2019-07-10,208.990,205.650
2019-07-11,212.590,206.000
2019-07-12,214.190,210.790
2019-07-15,214.600,209.790
2019-07-16,217.330,212.550
2019-07-17,215.775,213.160


In [38]:
def iterate_high_low(df_hl:pd.DataFrame, rows:int, overlap=True):
  """iterate multi lines to get Max of High and Mimimum of Lows across multiple days

  df: DataFrame - required
  rows: int - lines of values to compare for Max/Min value, rows > 1
  overlap: boolean - True for rolling comparison, False for block comparison 
  """
  high = []
  low = []

  if overlap:
    for i in [df_hl.iloc[pos:pos+rows] for pos in range(0, len(df_hl)-1)]:
      high.append(max(i['High']))
      low.append(min(i['Low']))
  else:
    for i, g in df_hl.groupby(np.arange(len(df_hl)) // rows):
      high.append(g['High'].max())
      low.append(g['Low'].min())
  
  return high, low

In [39]:
hh, ll = iterate_high_low(df_polygon, 2)
ll[0:10]

[204.705,
 203.54,
 203.54,
 205.65,
 206.0,
 209.79,
 209.79,
 212.55,
 211.75,
 211.75]

In [40]:
df_lh = pd.DataFrame()
df_lh['low'] = ll
df_lh['high'] = hh
df_lh

,low,high
0,204.7050,208.7600
1,203.5400,207.9400
2,203.5400,208.9900
3,205.6500,212.5900
4,206.0000,214.1900
...,...,...
498,363.8600,371.1499
499,363.8600,378.0900
500,370.7495,380.1100
501,370.9000,381.6400


In [41]:
# consider using a lower percentiles mean
multiday_volatility = (df_lh['high'] - df_lh['low'])/df_lh['low']
initial_multiday_pertentage = np.percentile(multiday_volatility, [60, 55, 50]).mean()
initial_multiday_pertentage

0.03641308359654272

Rate determination <br>
6 possibilities

1.   upper < low  (price is over the band)
2.   lower > high (price is below the band)
3.   high > upper > low > lower (price cross upper band)
4.   upper > high > lower > low (price cross lower band)
5.   upper > high & low > lower (price inside the band)
6.   high > upper & lower > low (price so volatile, cross the whole band)



In [42]:
# prepare the dataframe
df_ratio = df_hl.dropna()
df_ratio["UPPER"] = UPPER
df_ratio["LOWER"] = LOWER
df_ratio.dropna(inplace=True)
#calculate the ratio
rate1 = np.where(df_ratio["UPPER"] < df_ratio["Low"],initial_multiday_pertentage*2 , 0)
rate2 = np.where(df_ratio["LOWER"] > df_ratio["High"],initial_multiday_pertentage*-2 , 0)

# df_ratio_temp.query("ratio2 == ratio2") # to hack the fact that value == NaN will always be false


In [43]:
rate1 = np.where(df_ratio["UPPER"] < df_ratio["Low"],initial_multiday_pertentage*2 , 0) # extreme uptick
rate2 = np.where(df_ratio["LOWER"] > df_ratio["High"],initial_multiday_pertentage*-2 , 0) # extreme downward
rate3 = np.where((df_ratio['High'] > df_ratio['UPPER']) & (df_ratio['UPPER'] >  df_ratio['Low'] ) & (df_ratio['LOWER'] <  df_ratio['Low'] ), initial_multiday_pertentage*1.5, 0) # start to go up
rate4 = np.where((df_ratio['UPPER'] > df_ratio['High']) & (df_ratio['High'] > df_ratio['LOWER']) & (df_ratio['LOWER'] > df_ratio['Low'] ), initial_multiday_pertentage*-1.5, 0) # start to go down
# rate = np.vstack((rate1, rate2))
rate5 = np.where((df_ratio['UPPER'] > df_ratio['High']) & (df_ratio['Low'] > df_ratio['LOWER']), initial_multiday_pertentage, 0)
rate6 = np.where((df_ratio['High'] > df_ratio['UPPER']) & (df_ratio['LOWER'] > df_ratio['Low']), initial_multiday_pertentage, 0)
rate = rate1 + rate2 + rate3 + rate4 + rate5 +rate6

In [44]:
df_ratio['rate'] = rate
df_ratio

,High,Low,UPPER,LOWER,rate
Date,,,,,
2019-08-27,200.8595,196.5000,213.1037,194.9398,0.0364
2019-08-28,201.4700,196.2500,213.2419,194.1520,0.0364
2019-08-29,204.3700,201.3600,213.5492,193.7979,0.0364
2019-08-30,205.2100,203.0100,213.7822,193.6454,0.0364
2019-09-03,201.6100,196.5800,213.4460,193.1529,0.0364
...,...,...,...,...,...
2021-06-28,368.8700,363.8600,379.2004,354.8262,0.0364
2021-06-29,378.0900,370.7495,378.7209,356.3728,0.0364
2021-06-30,380.1100,370.9000,378.4697,358.1184,0.0546


In [45]:
# initial values
flag = 1
buy_signal = [np.nan]; sell_signal = [np.nan]
buyline = [np.nan]; sellline = [np.nan]
ceiling = 0 
bottom = df_ratio['Low'][0]

for i in range(len(df_ratio)-1):
    if flag == 1: # seeking buying oppotunity
        sell_signal.append(np.nan)
        sellline.append(np.nan)
        buyline.append(bottom * (1 + df_ratio['rate'][i]))
        bottom = min(bottom, df_ratio['Low'][i+1])
        if df_ratio["High"][i+1] > buyline[-1] > df_ratio["Low"][i+1]: # order hit the next day
            flag = -1
            buy_signal.append(buyline[-1])
            ceiling = df_ratio["High"][i+1]
        else:
            buy_signal.append(np.nan)


    if flag == -1: # seeking selling oppotunity
        buy_signal.append(np.nan)
        buyline.append(np.nan)
        sellline.append(ceiling * (1 - df_ratio['rate'][i]))
        ceiling = max(ceiling, df_ratio['High'][i+1])
        if df_ratio["High"][i+1] > sellline[-1] > df_ratio["Low"][i+1]: # sell hit the next day
            flag = 1
            sell_signal.append(sellline[-1])
            bottom = df_ratio["Low"][i+1]
        else:
            sell_signal.append(np.nan)




In [50]:
len(df_ratio)

467

In [47]:
df_ratio['buy line'] = buyline
df_ratio['sell line'] = sellline
df_ratio['buy signal'] = buy_signal
df_ratio['sell signal'] = sell_signal
df_ratio

ValueError: Length of values (478) does not match length of index (467)